## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pytz
import glob
import pathlib

this_dir = pathlib.Path(os.path.abspath(""))
data_dir = this_dir / "data"

In [3]:
import requests
import pandas as pd
import json
from datetime import datetime

## Download

Retrieve the page

In [4]:
url = "https://services2.arcgis.com/bYBANhmQGwSSLC0l/arcgis/rest/services/AllZipCodePoints___TEST_TEST/FeatureServer/0//query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [5]:
r = requests.get(url)

In [6]:
data = r.json()

## Parse

In [7]:
dict_list = []

In [8]:
for item in data["features"]:
    zip_code = item["attributes"]["ZIP_CODE"]
    city = item["attributes"]["City"]
    d = dict(
        county="Tulare",
        area=zip_code,
        zip_code=zip_code,
        city=city,
        confirmed_cases=item["attributes"]["Cases"],
    )
    dict_list.append(d)

In [9]:
df = pd.DataFrame(dict_list)

Clean names

In [10]:
df.loc[df.zip_code == "93212", "city"] = "Corcoran, Angiola"
df.loc[df.zip_code == "93646", "city"] = "Traver"
df.loc[df.zip_code == "93654", "city"] = "Orosi"

In [11]:
df["area"] = df["city"] + ": " + df["zip_code"]

Get timestamp

In [12]:
date_url = "https://services2.arcgis.com/bYBANhmQGwSSLC0l/arcgis/rest/services/ZipCodeAreas__Test/FeatureServer/0?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [13]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [14]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [15]:
latest_date = pd.to_datetime(timestamp).date()

In [16]:
df["county_date"] = latest_date

Prep for export

In [17]:
export_df = df[["county", "area", "confirmed_cases", "county_date", "zip_code"]].rename(
    columns={"zip_code": "zip"}
)

In [18]:
export_df = export_df.dropna(axis=0, subset=["area"])

In [19]:
export_df.loc[(export_df.area == "TerraBella: 93270"), "area"] = "Terra Bella: 93270"

In [20]:
len(export_df.area)

28

## Vet

In [21]:
try:
    assert not len(export_df) > 28
except AssertionError:
    raise AssertionError("Tulare County's scraper has extra rows")

In [22]:
try:
    assert not len(export_df) < 28
except AssertionError:
    raise AssertionError("Tulare County's scraper is missing rows")

## Export

Set date

In [23]:
tz = pytz.timezone("America/Los_Angeles")

In [24]:
today = datetime.now(tz).date()

In [25]:
slug = "tulare"

In [26]:
export_df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [27]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [28]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"])
        df["date"] = file_date
    df_list.append(df)

In [29]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [30]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)

Set date

In [31]:
now = env.get_today()

NameError: name 'env' is not defined

In [ ]:
writer.to_csv(export_df, f"places-scrapers/tulare/{now}.csv")

In [ ]:
export_df